<a href="https://colab.research.google.com/github/MarceloMena/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/SEMINARIO/David_Mena_SE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:   David Marcelo Mena Ruiz
Url: https://colab.research.google.com/drive/1oiisb9CRRhCSwPXEPqpEriWXxx-yi5tO<br>
Problema:
> Organizar los horarios de partidos de La Liga<br>

Descripción del problema:(copiar enunciado)

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos
de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un
algoritmo que realice la asignación de los partidos a los horarios de forma que maximice
la audiencia.




                                        

In [0]:
import numpy as np
import pandas as pd
import random
import copy
import itertools
import math

In [0]:
EQUIPOS =['Celta','Real Madrid','Valencia','Real Sociedad',
         'Mallorca','Eibar','Athletic', 'Barcelona',
         'Leganes','Osasuna','Villareal','Granada',
         'Alaves','Levante','Espanyol','Sevilla',
         'Valladolid','Betis','Atletico','Getafe']

CATEGORIAS = {'CAT_A': ['Celta','Real Madrid','Valencia','Real Sociedad'],
             'CAT_B': ['Mallorca','Eibar','Athletic', 'Barcelona',
                       'Leganes','Osasuna','Villareal','Granada',
                       'Alaves','Levante'],
             'CAT_C': ['Espanyol','Sevilla','Valladolid','Betis','Atletico','Getafe']}

In [0]:
## algoritmo generador de partidos, une dos equipos de manera ##
## aleatoria hasta que todos estan asignados en diferentes partidos ##
## solo puede asignarse un equipo una vez en la jornada ##

def generar_partidos(EQUIP):
    random.shuffle(EQUIP)
    HOR = []
    i=0
    while 2*len(HOR) < len(EQUIP) :
        N_PART =  2*i
        A = EQUIP[N_PART]
        B = EQUIP[N_PART+1]
        C = (A,B)
        HOR.append(C)
        i += 1
    return HOR

PARTIDOS = generar_partidos(EQUIPOS)
PARTIDOS


## Como entrada recibe los partidos generados anteriormente ##
## y salida retorna los valores de audiencia de cada partido ##
## Calcula la audiencia de cada partidos dependiendo de ##
## de las categorias de los equipos A, B o C

def audiencia(N):
    HOR = N
    AUDIENCIA = []
    for i in range(len(HOR)):
        equipo_1, equipo_2 = HOR[i]
        cat_a = CATEGORIAS.get('CAT_A')
        cat_b = CATEGORIAS.get('CAT_B')
        cat_c = CATEGORIAS.get('CAT_C')
        if equipo_1 in cat_a:
            if equipo_2 in cat_a:
                AUDIENCIA.append(2)
            if equipo_2 in cat_b:
                AUDIENCIA.append(1.3)
            if equipo_2 in cat_c:
                AUDIENCIA.append(1)
        if equipo_1 in cat_b:
            if equipo_2 in cat_a:
                AUDIENCIA.append(1.3)
            if equipo_2 in cat_b:
                AUDIENCIA.append(0.9)
            if equipo_2 in cat_c:
                AUDIENCIA.append(0.75)
        if equipo_1 in cat_c:
            if equipo_2 in cat_a:
                AUDIENCIA.append(1)
            if equipo_2 in cat_b:
                AUDIENCIA.append(0.75)
            if equipo_2 in cat_c:
                AUDIENCIA.append(0.47)
    return AUDIENCIA

AUDIENCIA_POR_CATEGORIA = audiencia(PARTIDOS)

### Calculo de solucion con fuerza bruta ###
### Al ser el numero de soluciones sin repeticion de horario ##

COSTES = AUDIENCIA_POR_CATEGORIA

def fuerza_bruta(COSTES):
    mejor_valor = 0
    mejor_solucion = ''
    PFB = list(itertools.permutations(COSTES))
    PARFB = list(itertools.permutations(range(len(COSTES))))
    for s in range(len(PFB)):
        valor_tmp = 0.4*PFB[s][0] + 0.4*PFB[s][1] + 0.75*PFB[s][2] + 0.8*PFB[s][3] + PFB[s][4] + 0.55*PFB[s][5] + 0.7*PFB[s][6] + 0.5*PFB[s][7] + PFB[s][8] + 0.4*PFB[s][9]
        if valor_tmp > mejor_valor:
            mejor_solucion = PARFB[s]
            mejor_valor = valor_tmp
    print('la mejor solucion por fuerza bruta es: ',  mejor_solucion, '\ncon una audiencia de: ', mejor_valor, "\n")
    return mejor_solucion, mejor_valor

### Calculo de Auidiencia tomando en cuenta una lista que ###
### contiene los vectores de cada uno de los horarios ###

def audiencia_final(M):
    V20,S12,S16,S18,S20,D12,D16,D18,D20,L20 = M
    AUDIENCIA_JORNADA = 0.4*calculador_audiencia(V20) + 0.4*calculador_audiencia(S12) + 0.75*calculador_audiencia(S16) + 0.8*calculador_audiencia(S18) + calculador_audiencia(S20) + 0.55*calculador_audiencia(D12) + 0.7*calculador_audiencia(D16) + 0.5*calculador_audiencia(D18) + calculador_audiencia(D20) + 0.4*calculador_audiencia(L20)
    return(AUDIENCIA_JORNADA)

### Calculo de la audiencia para la primera corrida del ###
### algoritmo genetico donde se genera la primera poblacion  aleatoria ###

def calculador_audiencia_jornada(JORNADA):
    V20,S12,S16,S18,S20,D12,D16,D18,D20,L20 = JORNADA
    AUDIENCIA_JORNADA = 0.4*sum(V20) + 0.4*sum(S12) + 0.75*sum(S16) + 0.8*sum(S18) + sum(S20) + 0.55*sum(D12) + 0.7*sum(D16) + 0.5*sum(D18) + sum(D20) + 0.4*sum(L20)
    return(AUDIENCIA_JORNADA)

### Calculo de la audiencia final, teniendo un vector de    ###
### posiciones de cada partido, y un vector donde estan las ###
### audiencias dependiendo de las categorias de los equipos ###

def calculo_final(SOLU_PARC,AUDIENCIA_POR_CATEGORIA):
    JORNADA = ordenador_por_jornada(SOLU_PARC,AUDIENCIA_POR_CATEGORIA)
    AUDIENCIA_FINAL = calculador_audiencia_jornada(JORNADA)
    return(AUDIENCIA_FINAL)

## Asigna los horarios de los partidos de manera aleatoria ##
## entrada son los partidos definidos en el generador_partidos ##
## para la primera parte del algoritmo genetico ##

def asignacion_horarios(HOR):
    V20 = []
    S12 = []
    S16 = []
    S18 = []
    S20 = []
    D12 = []
    D16 = []
    D18 = []
    D20 = []
    L20 = []
    POSICION = []
    N = len(HOR)
    PROGRAMACION = list(np.zeros(N))
    for i in range(N):
        ind_al = random.randint(0,9)
        POSICION.append(ind_al)
        if ind_al == 0:
            V20.append(HOR[i])
        if ind_al == 1:
            S12.append(HOR[i])
        if ind_al == 2:
            S16.append(HOR[i])
        if ind_al == 3:
            S18.append(HOR[i])
        if ind_al == 4:
            S20.append(HOR[i])
        if ind_al == 5:
            D12.append(HOR[i])
        if ind_al == 6:
            D16.append(HOR[i])
        if ind_al == 7:
            D18.append(HOR[i])
        if ind_al == 8:
            D20.append(HOR[i])
        if ind_al == 9:
            L20.append(HOR[i])
    JORNADA = [V20,S12,S16,S18,S20,D12,D16,D18,D20,L20]
    return JORNADA,POSICION

## Realiza la operacion ya con el vector de posicion generado ###
## con los hijos del algoritmo genetico ##

def ordenador_por_jornada(HIJA,AUDIENCIA_HORARIO):
    HORARIOS_PARTIDOS = []
    V20 = []
    S12 = []
    S16 = []
    S18 = []
    S20 = []
    D12 = []
    D16 = []
    D18 = []
    D20 = []
    L20 = []
    for i in range(len(HIJA)):
        if HIJA[i] == 0:
            V20.append(AUDIENCIA_HORARIO[i])
        if HIJA[i] == 1:
            S12.append(AUDIENCIA_HORARIO[i])
        if HIJA[i] == 2:
            S16.append(AUDIENCIA_HORARIO[i])
        if HIJA[i] == 3:
            S18.append(AUDIENCIA_HORARIO[i])
        if HIJA[i] == 4:
            S20.append(AUDIENCIA_HORARIO[i])
        if HIJA[i] == 5:
            D12.append(AUDIENCIA_HORARIO[i])
        if HIJA[i] == 6:
            D16.append(AUDIENCIA_HORARIO[i])
        if HIJA[i] == 7:
            D18.append(AUDIENCIA_HORARIO[i])
        if HIJA[i] == 8:
            D20.append(AUDIENCIA_HORARIO[i])
        if HIJA[i] == 9:
            L20.append(AUDIENCIA_HORARIO[i])
    JORNADA = [V20,S12,S16,S18,S20,D12,D16,D18,D20,L20]
    return JORNADA
  
## funcion que calcula el porcentaje deacuerdo al numero de ##
## coincidencias dependiendo del numero de partidos en el mismo horario ##

def coincidencias(N):
    NUM_PARTIDOS = len(N)
    if NUM_PARTIDOS == 0:
        porc_dismin = 1
    if NUM_PARTIDOS == 1:
        porc_dismin = 0.75
    if NUM_PARTIDOS == 2:
        porc_dismin = 0.55
    if NUM_PARTIDOS == 3:
        porc_dismin = 0.40
    if NUM_PARTIDOS == 4:
        porc_dismin = 0.3
    if NUM_PARTIDOS == 5:
        porc_dismin = 0.25
    if NUM_PARTIDOS == 6:
        porc_dismin = 0.22
    if NUM_PARTIDOS == 7:
        porc_dismin = 0.2
    if NUM_PARTIDOS == 8:
        porc_dismin = 0.2
    return porc_dismin

## Calcula la audiencia total en cada partido con el nummero de coincidencias ##
## antes de ser introducido a los factores de horario
def calculador_audiencia(X):
    N = audiencia(X)
    SUMA_AUDIENCIAS = sum(N)
    FACT_COIN = coincidencias(N)
    AUD_HOR = SUMA_AUDIENCIAS*coincidencias(N)
    return AUD_HOR 
  
## Generador de cruce en una pablacion a medida que se genera una descendencia ##

def Cruzar(poblacion,mutacion, VECT_POS):
    poblacion_copia = copy.deepcopy(poblacion)
    poblacion_final = copy.deepcopy(poblacion)
    while len(poblacion_copia) > 1:  
        padre1,padre2 = random.sample(poblacion_copia   ,2)
        poblacion_copia.remove(padre1)
        poblacion_copia.remove(padre2)
        poblacion_final.extend(Descendencia([padre1,padre2],VECT_POS,mutacion))
    return poblacion_final

## Calculador de descendecia con dos padres cortados en dos puntos aleatorios ##
## introduce la mutacion de los hijos ##
def Descendencia(padres, problem, mutacion):
    pc = random.sample( range(1,len(padres[0])),1)[0]
    hijo1 =  padres[0][:pc] + padres[1][pc:]
    hijo2 =  padres[1][:pc] + padres[0][pc:]
    return [hijo1,hijo2,Mutar(hijo1,mutacion),Mutar(hijo2,mutacion)]

## Mutacion de los hijos de acuerdo a un generador aleatorio donde se ##
## intercambian las posiciones de dos valores en posiciones aleatorias ##
def Mutar(solucion,mutacion):
    if random.random() < mutacion:
        sel1,sel2 = sorted(random.sample(set(solucion)-{solucion[0]},2))
        return solucion[:sel1] + [solucion[sel2]] + solucion[sel1+1:sel2] + [solucion[sel1]]  + solucion[sel2+1:] 
    else:
        return solucion[::] 
      
## Se seleccionan los hijos y padres que pasaran a la segunda generacion para ##
## su evaluacion de acuerdo a la funcion de elitismo y el numero total de la ##
## poblacion ordenados de mayor a menor ##
    
def Seleccionar(problem,poblacion, N, elitismo, AUDIENCIA_POR_CATEGORIA):
    poblacion_ordenada = sorted([ [calculo_final(problem, AUDIENCIA_POR_CATEGORIA), problem] for problem in poblacion ], key= lambda x:x[0] )
    return [x[1] for x in poblacion_ordenada][:int(N*elitismo)]  + random.sample([x[1] for x in poblacion_ordenada][int(N*elitismo):] , int(N*(1-elitismo))) 
  
## Evaluacion de toda la poblacion generada para encontrar la mejor solcion de ##
## cada generacion y asi almacenarla para comparar en la siguiente generacion ##
def evaluar_poblacion(poblacion, AUDIENCIA_POR_CATEGORIA,MEJOR_AUDIENCIA,MEJOR_HORARIO):
    for i in range(len(poblacion)):
        audiencia_hijo = calculo_final(poblacion[i], AUDIENCIA_POR_CATEGORIA)
        horario_hijo = poblacion[i]
        mejor_horario = MEJOR_HORARIO
        mejor_audiencia = MEJOR_AUDIENCIA
        if audiencia_hijo > mejor_audiencia:
            mejor_audiencia = audiencia_hijo
            mejor_horario = horario_hijo
    return mejor_audiencia, mejor_horario

## algoritmo genetico que como entrada requiere##
## PARTIDOS: partidos generados con funcion generador_partidos
## N: Poblacion
## GENERACIONES: numero de generaciones a realizar
## mutacion: porcentaje de mutaciones a realizar por cada hijo
## fitness: elitismo entre generaciones

def algoritmo_genetico(PARTIDOS, N, GENERACIONES, mutacion, fitness):
    PADRES = []
    VECT_POS = []
    VECT_AUD = []
    VECT_HOR = []
    AUDIENCIA_POR_CATEGORIA = audiencia(PARTIDOS)
    MEJOR_HORARIO,POSICION = asignacion_horarios(PARTIDOS)
    MEJOR_AUDIENCIA = audiencia_final(MEJOR_HORARIO)
    # Generador de poblacion inicial
    for i in range(N):
        HORARIO_GENERADO, POSICION = asignacion_horarios(PARTIDOS)
        AUDIENCIA_HORARIO = audiencia_final(HORARIO_GENERADO)
        CONCATENADO = (HORARIO_GENERADO,POSICION,AUDIENCIA_HORARIO)
        PADRES.append(CONCATENADO)
        VECT_POS.append(POSICION) #lista que almacena las jornadas generadas
        VECT_AUD.append(AUDIENCIA_HORARIO)
        VECT_HOR.append(HORARIO_GENERADO)
        # Busqueda de mejor horario y mejor audiencia #
        # en primera poblacion #
        if AUDIENCIA_HORARIO > MEJOR_AUDIENCIA:
            MEJOR_HORARIO = HORARIO_GENERADO
            MEJOR_AUDIENCIA = AUDIENCIA_HORARIO
    # Generaciones
    for i in range(GENERACIONES):
        poblacion = Cruzar(VECT_POS, mutacion, PARTIDOS) #Genera los hijos
        # Selecciona de todos los hijos y padres segun funcion de elitismo #
        # que tengan una mayor audiencia y los selecciona de manera que en su#
        # poblacion no exceda la poblacion solicitada inicalmente #
        poblacion = Seleccionar(VECT_POS,VECT_POS, N, fitness,AUDIENCIA_POR_CATEGORIA) 
        # Busqueda de la mejor solucion y la mejor audiencia en cada generacion #
        # la mejor solucion para poder comparar en las futuras generaciones #
        MEJOR_AUDIENCIA, MEJOR_HORARIO = evaluar_poblacion(poblacion, AUDIENCIA_POR_CATEGORIA,MEJOR_AUDIENCIA,MEJOR_HORARIO)
  
    print("Generacion #", i, "\nLa mejor solución es:" , MEJOR_HORARIO, "\ncon audiencia " , MEJOR_AUDIENCIA, "\n")   
    return MEJOR_AUDIENCIA, MEJOR_HORARIO


# Funcion que nos permite traducir los horarios por partido a un horario #
# con los partidos que se jugaran por dia #
def presentador(MEJOR_AUDIENCIA, MEJOR_HORARIO,PARTIDOS):
  JORNADA = ordenador_por_jornada(MEJOR_HORARIO,PARTIDOS)
  for i in range(len(JORNADA)):
      if i == 0:
          print('Viernes 20h: ', JORNADA[i])
      if i == 1:
          print('Sabado 12h: ', JORNADA[i])
      if i == 2:
          print('Sabado 16h: ', JORNADA[i])
      if i == 3:
          print('Sabado 18h: ', JORNADA[i])
      if i == 4:
          print('Sabado 20h: ', JORNADA[i])
      if i == 5:
          print('Domingo 12h: ', JORNADA[i])
      if i == 6:
          print('Domingo 16h:: ', JORNADA[i])
      if i == 7:
          print('Domingo 18h: ', JORNADA[i])
      if i == 8:
          print('Domingo 20h: ', JORNADA[i])
      if i == 9:
          print('Lunes 20h: ', JORNADA[i])
  print('La audiencia de la jornada es (Millones de televidentes): ', float(MEJOR_AUDIENCIA) )  




In [4]:
mejor_solucion, mejor_valor = fuerza_bruta(AUDIENCIA_POR_CATEGORIA)
presentador(mejor_valor, mejor_solucion,PARTIDOS)

la mejor solucion por fuerza bruta es:  (1, 9, 2, 0, 8, 3, 6, 4, 5, 7) 
con una audiencia de:  7.2205 

Viernes 20h:  [('Valladolid', 'Barcelona')]
Sabado 12h:  [('Alaves', 'Celta')]
Sabado 16h:  [('Eibar', 'Osasuna')]
Sabado 18h:  [('Valencia', 'Real Sociedad')]
Sabado 20h:  [('Atletico', 'Levante')]
Domingo 12h:  [('Real Madrid', 'Athletic')]
Domingo 16h::  [('Leganes', 'Granada')]
Domingo 18h:  [('Getafe', 'Sevilla')]
Domingo 20h:  [('Espanyol', 'Villareal')]
Lunes 20h:  [('Betis', 'Mallorca')]
La audiencia de la jornada es (Millones de televidentes):  7.2205


In [5]:
MEJOR_AUDIENCIA, MEJOR_HORARIO=algoritmo_genetico(PARTIDOS,N = 100, GENERACIONES = 200, mutacion = 0.35, fitness = 0.2)
presentador(MEJOR_AUDIENCIA, MEJOR_HORARIO,PARTIDOS)

Generacion # 199 
La mejor solución es: [5, 6, 5, 8, 2, 8, 4, 4, 8, 9] 
con audiencia  8.185500000000001 

Viernes 20h:  []
Sabado 12h:  []
Sabado 16h:  [('Espanyol', 'Villareal')]
Sabado 18h:  []
Sabado 20h:  [('Leganes', 'Granada'), ('Atletico', 'Levante')]
Domingo 12h:  [('Alaves', 'Celta'), ('Eibar', 'Osasuna')]
Domingo 16h::  [('Betis', 'Mallorca')]
Domingo 18h:  []
Domingo 20h:  [('Valladolid', 'Barcelona'), ('Valencia', 'Real Sociedad'), ('Real Madrid', 'Athletic')]
Lunes 20h:  [('Getafe', 'Sevilla')]
La audiencia de la jornada es (Millones de televidentes):  8.185500000000001


(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




*Respuest*a

**La unica restriccion que nos cambia el numero de opciones es si es permitido o no que varios partidos coincidan en el mismo horario**

In [15]:
print('numero de posibilidades si no existe la repeticion de horarios: ', math.factorial(10))

numero de posibilidades si no existe la repeticion de horarios:  3628800


In [16]:
print('numero de posibilidades si existe la repeticion de horarios: ', 10**10)

numero de posibilidades si existe la repeticion de horarios:  10000000000


Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

La mejor opcion que encontre fue utilizando un vector del siguiente tipo:

SOLUCION = [P1,P2,...,P3] 

**En donde la posicion SOLUCION[0] equivale al primer partido y el valor de P1 es el horario asignado de la siguiente manera:**

0 = Viernes 20h<br>
1 = Sabado 12h<br>
2 = Sabado 16h<br>
3 = Sabado 18h<br>
4 = Sabado 20h<br>
5 = Domingo 12h<br>
6 = Domingo 16h<br>
7 = Domingo 18h<br>
8 = Domingo 20h<br>
9 = Lunes 20h<br>

Esta manera de trabajar me permitio implementar mas facilmente la coincidencia de partidos en el mismo horario, tomando en cuenta que para el algoritmo genetico era mas sencillo trabajar con cadenas de valores del mismo tamanno y que se podria asignar el horario con un valor del 0 al 9

En un principio utilice el horario como posicion y los valores del vector como el partido a llevarse a cabo, pero se me dificulto demasiado el calculo cuando quise implementar, varios partidos en un solo horario

In [17]:
print('Vector con los horarios de cada partido con la mejor solucion en algoritmo genetico: ', MEJOR_HORARIO)

Vector con los horarios de cada partido con la mejor solucion en algoritmo genetico:  [6, 2, 3, 8, 4, 4, 4, 7, 2, 8]


Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

**El problema es uno de maximizacion y mi funcion objetivo era la definida como calculo_final(SOLU_PARC,AUDIENCIA_POR_CATEGORIA).** <br>

    AUDIENCIA_JORNADA = 0.4*calculador_audiencia(V20) + 0.4*calculador_audiencia(S12) + 0.75*calculador_audiencia(S16) + 0.8*calculador_audiencia(S18) + calculador_audiencia(S20) + 0.55*calculador_audiencia(D12) + 0.7*calculador_audiencia(D16) + 0.5*calculador_audiencia(D18) + calculador_audiencia(D20) + 0.4*calculador_audiencia(L20)


Como entradas para esta funcion se encuentran:<br>
SOLU_PARC: vector de partidos y horarios <br>
AUDIENCIA_POR_CATEGORIA : calculo de audiencia en cada uno de los partidos de acuerdo a su categoria<br>

In [9]:
print('Calculo de audiencia: ', calculo_final(MEJOR_HORARIO,AUDIENCIA_POR_CATEGORIA))

Calculo de audiencia:  8.185500000000001


Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta


Utilizar fuerza bruta lo considere unicamente factible para el caso donde no se permitian las repeticiones, debido a que el orden del problema no iba a permitir realizar en un tiempo razonable el caso donde si se permitian las repeticiones

In [0]:
## la unica entrada del algoritmo son los costes o la audiencia ##
## esta se obtiene dependiendo de las categorias de cada equipo ##
## para cada uno de los partidos ##

def fuerza_bruta(COSTES):
    mejor_valor = 0
    mejor_solucion = ''
    PFB = list(itertools.permutations(COSTES))
    PARFB = list(itertools.permutations(range(len(COSTES))))
    for s in range(len(PFB)):
        valor_tmp = 0.4*PFB[s][0] + 0.4*PFB[s][1] + 0.75*PFB[s][2] + 0.8*PFB[s][3] + PFB[s][4] + 0.55*PFB[s][5] + 0.7*PFB[s][6] + 0.5*PFB[s][7] + PFB[s][8] + 0.4*PFB[s][9]
        if valor_tmp > mejor_valor:
            mejor_solucion = PARFB[s]
            mejor_valor = valor_tmp
    print('la mejor solucion por fuerza bruta es: ',  mejor_solucion, '\ncon una audiencia de: ', mejor_valor, "\n")
    return mejor_solucion, mejor_valor

In [11]:
mejor_solucion, mejor_valor = fuerza_bruta(AUDIENCIA_POR_CATEGORIA)
presentador(mejor_valor, mejor_solucion,PARTIDOS)

la mejor solucion por fuerza bruta es:  (1, 9, 2, 0, 8, 3, 6, 4, 5, 7) 
con una audiencia de:  7.2205 

Viernes 20h:  [('Valladolid', 'Barcelona')]
Sabado 12h:  [('Alaves', 'Celta')]
Sabado 16h:  [('Eibar', 'Osasuna')]
Sabado 18h:  [('Valencia', 'Real Sociedad')]
Sabado 20h:  [('Atletico', 'Levante')]
Domingo 12h:  [('Real Madrid', 'Athletic')]
Domingo 16h::  [('Leganes', 'Granada')]
Domingo 18h:  [('Getafe', 'Sevilla')]
Domingo 20h:  [('Espanyol', 'Villareal')]
Lunes 20h:  [('Betis', 'Mallorca')]
La audiencia de la jornada es (Millones de televidentes):  7.2205


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

**la complejidad es O(n!) tomando en cuenta que debemos de hacer un numero de permutaciones de todos los equipos para validar todas las combinaciones de partidos en los diferentes horarios**

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

Utilizo un algoritmo genetico para la resolucion del problema debido a la clara reduccion de la complejidad del algoritmo y debido al numero de posibilidades que se tenia, tomando en cuenta que ahora estoy tratando de encontrar **el numero de soluciones permitiendo la repeticion de horarios de los partidos**, al introducir una tecnica heuristica no obtendre un dato exacto de la solucion pero si una muy cercana a ella

In [0]:
def algoritmo_genetico(PARTIDOS, N, GENERACIONES, mutacion, fitness):
    # Generacion de padres #
    PADRES = []
    VECT_POS = []
    VECT_AUD = []
    VECT_HOR = []
    AUDIENCIA_POR_CATEGORIA = audiencia(PARTIDOS)
    MEJOR_HORARIO,POSICION = asignacion_horarios(PARTIDOS)
    MEJOR_AUDIENCIA = audiencia_final(MEJOR_HORARIO)
    for i in range(N):
        HORARIO_GENERADO, POSICION = asignacion_horarios(PARTIDOS)
        AUDIENCIA_HORARIO = audiencia_final(HORARIO_GENERADO)
        CONCATENADO = (HORARIO_GENERADO,POSICION,AUDIENCIA_HORARIO)
        PADRES.append(CONCATENADO)
        VECT_POS.append(POSICION)
        VECT_AUD.append(AUDIENCIA_HORARIO)
        VECT_HOR.append(HORARIO_GENERADO)
        # Busqueda de mejor horario y mejor audiencia #
        if AUDIENCIA_HORARIO > MEJOR_AUDIENCIA:
            MEJOR_HORARIO = HORARIO_GENERADO
            MEJOR_AUDIENCIA = AUDIENCIA_HORARIO
    # Generador de hijos #
    #Inciamos el cliclo de generaciones
    for i in range(GENERACIONES):
    
        #Cruce de la poblacion(incluye mutación)
        poblacion = Cruzar(VECT_POS, mutacion, PARTIDOS)

        #Seleccionamos la población
        poblacion = Seleccionar(VECT_POS,VECT_POS, N, fitness,AUDIENCIA_POR_CATEGORIA)

        #Evaluamos la nueva población
        MEJOR_AUDIENCIA, MEJOR_HORARIO = evaluar_poblacion(poblacion, AUDIENCIA_POR_CATEGORIA,MEJOR_AUDIENCIA,MEJOR_HORARIO)
  
    print("Generacion #", i, "\nLa mejor solución es:" , MEJOR_HORARIO, "\ncon audiencia " , MEJOR_AUDIENCIA, "\n")

    #Numero de generaciones. Criterio de parada
    
    return MEJOR_AUDIENCIA, MEJOR_HORARIO

(*)Calcula la complejidad del algoritmo 

Respuesta

**La complejidad del algoritmo es O(n^2), siendo n el numero de su poblacion**

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

**Debido a que he utilizado un algoritmo genetico es imprecindible generar una poblacion aleatoria de entrada, por la manera en que he disennado el algoritmo es complicado la visualizacion de los datos de enrada aleatorios, en el inicio del programa explico como se generan estan poblaciones y se evaluan**

Aplica el algoritmo al juego de datos generado

Respuesta

In [13]:
MEJOR_AUDIENCIA, MEJOR_HORARIO=algoritmo_genetico(PARTIDOS,N = 100, GENERACIONES = 200, mutacion = 0.35, fitness = 0.2)
presentador(MEJOR_AUDIENCIA, MEJOR_HORARIO,PARTIDOS)

Generacion # 199 
La mejor solución es: [6, 2, 3, 8, 4, 4, 4, 7, 2, 8] 
con audiencia  8.4125 

Viernes 20h:  []
Sabado 12h:  []
Sabado 16h:  [('Betis', 'Mallorca'), ('Real Madrid', 'Athletic')]
Sabado 18h:  [('Eibar', 'Osasuna')]
Sabado 20h:  [('Espanyol', 'Villareal'), ('Valencia', 'Real Sociedad'), ('Leganes', 'Granada')]
Domingo 12h:  []
Domingo 16h::  [('Alaves', 'Celta')]
Domingo 18h:  [('Atletico', 'Levante')]
Domingo 20h:  [('Valladolid', 'Barcelona'), ('Getafe', 'Sevilla')]
Lunes 20h:  []
La audiencia de la jornada es (Millones de televidentes):  8.4125


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

*Ninguna*

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta